In [ ]:
import numpy as np
import scipy as scp
import pylab as pyl
import matplotlib.pyplot as plt
import scipy.io as sio
import pandas as pd
import holoviews as hv
import panel as pn
import time
from panel.pane import LaTeX
hv.extension('bokeh')
from PIL import Image
from io import BytesIO
import requests

import os
from PIL import Image

# Chargement et affichage de l'image de base

In [ ]:
!unzip images.zip

In [ ]:
#chemin_image =  "/content/margot_pdp.jpg"
#chemin_image =  "/content/teckel2.jpg"
#chemin_image = "/content/carrelage.jpg"
chemin_image = "./lena.jpg"
image = Image.open(chemin_image)
image_nbw = image.convert("L")
image_nbw.save(chemin_image)
image

In [ ]:
def chargeData(name):
    if name == 'lena' :
        res=np.array(Image.open("./lena.jpg")).astype(float)
    if name == 'carrelage' :
        res=np.array(Image.open("./carrelage.jpg")).astype(float)
    if name=='Mur_brique':
        res=np.array(Image.open("/content/Mur_brique.jpg")).astype(float)
    return res

def MSE(image1,image2):
    mse = np.mean(np.square(image1.astype(float) - image2.astype(float)))
    return mse

def PSNR(image1, image2, peak=255):
    mse = MSE(image1,image2)
    psnr = 10*np.log10(peak**2/mse)
    return psnr

In [ ]:
image=chargeData('lena')
imagesRef= {'carrelage' : image}
options = dict(cmap='gray',xaxis=None,yaxis=None,width=400,height=400,toolbar=None)
pn.Row(hv.Raster(image).opts(**options))

In [ ]:
print(f"L'image originale est de taille {np.shape(image)}")

# Reshape de l'image de base (optionnel)

On baisse la résolution de l'image pour accélerer les calculs.

In [ ]:
new_size =160,160
im_good=Image.fromarray(image)
im_good = im_good.resize(new_size)
im_good_array = np.asarray(im_good)
pn.Row(hv.Raster(im_good_array).opts(**options))

# Bruitage de l'image

In [ ]:
def create_gaussian_noise(img,mean,var):
    img = img/255
    result = np.copy(img)
    # Adding gaussian noise to the image
    gauss = np.random.normal(mean, var**0.5, img.shape)
    result = result + gauss
    result = np.clip(result, 0, 1)
    result = result*255
    return result

In [ ]:
im_noised_array = create_gaussian_noise(im_good_array,mean=0,var=0.005)
pn.Row(hv.Raster(im_noised_array).opts(**options))

In [ ]:
psnr=pd.DataFrame([PSNR(im_noised_array, im_good_array)],columns=['PSNR image de base / image bruitée'])
display(psnr)

# Débruitage

Plusieurs distances peuvent être utilisées pour comparer les patchs entre eux. La meilleure de celles que nous avons testées est la suivante : pour deux patchs $p_1,p_2$, on calcule la distance $d$ entre les deux avec la formule suivante :

$$
d(p_1,p_2)=e^{-||w \odot (p_1-p_2)^2||}
$$

Avec $w$ une matrice des poids associés à chaque pixel. 

In [ ]:
def patch_distance_2d(p1, p2, s, w):
    return np.exp(-np.multiply(w,(p1-p2)**2).sum())

In [ ]:
def big_patch(img, coord_x, coord_y, size_w): # Retourne les coordonnées inférieures de la grosse fenêtre de recherche de patchs (retourne les coordonnées de l'angle en haut à gauche).
    """
    img : image
    coord_x : coordonnée x
    coord_y : coordonnée y
    size_w : taille de la grosse fenêtre
    """
    # On calcule les coordonnées de la grosse fenêtre de recherche
    b_inf_x = coord_x - (size_w//2)
    b_sup_x = coord_x + (size_w//2)
    b_inf_y = coord_y - (size_w//2)
    b_sup_y = coord_y + (size_w//2)
    # On gère les cas ou on déborde de l'image
    if b_inf_x < 0 :
        b_sup_x -= b_inf_x
        b_inf_x = 0
    elif b_sup_x >= np.shape(img)[1] :
        b_inf_x -= b_sup_x - (np.shape(img)[1] - 1)
        b_sup_x = np.shape(img)[1] - 1
    if b_inf_y < 0 :
        b_sup_y -= b_inf_y
        b_inf_y = 0
    elif b_sup_y >= np.shape(img)[0] :
        b_inf_y -= b_sup_y - (np.shape(img)[0] - 1)
        b_sup_y = np.shape(img)[0] - 1
    return int(b_inf_x), int(b_inf_y)

def non_local_means_denoising(image,
                    small_window_size=7,
                    big_window_size=35,
                    h=40,
                    ): # Débruite une image par moyennes non locales
    """
    image : image à débruiter
    small_window_size : taille des petits patchs
    big_window_size : tailles des grosses fenêtres de recherche de patchs
    h : parametre influençant le calcul des distances entre les patchs
    """
    # On affiche le nombre total d'itérations
    iterator = 0
    total_iterations = np.shape(image)[1]*np.shape(image)[0]*(big_window_size - small_window_size + 1)**2
    print("TOTAL ITERATIONS = ", total_iterations)
    # Les tailles des fenêtres doivent être impaires pour qu'elles soient centrées sur un pixel
    if small_window_size % 2 == 0:
        small_window_size += 1
    if big_window_size % 2 == 0:
        big_window_size += 1
    # On définit les moitiés de patchs
    small_half_width = small_window_size // 2
    big_half_width = big_window_size // 2
    # On crée une image avec du padding sur les cotés
    padded_image = np.ascontiguousarray(
        np.pad(image, ((small_half_width, small_half_width), (small_half_width, small_half_width)),
               mode='reflect').astype(np.float64))
    # On crée l'image de sortie
    output_image = np.zeros_like(padded_image)
    # On crée les parametres de calculs de distances entre patchs
    t = small_window_size//2
    A = ((small_window_size - 1.) / 4.)
    range_vals = np.arange(-t, t + 1)
    xg_row, xg_col = np.meshgrid(range_vals, range_vals, indexing='ij')
    # On crée une matrice de poids w, qui pondère le calcul des distances entre patchs. Les pixels centraux des patchs sont ceux qui influencent le plus les distances entre patchs
    w = np.ascontiguousarray(np.exp(-(xg_row ** 2  + xg_col ** 2)/ (2 * A**2)))
    w *= 1. / (np.sum(w) * h**2)
    # On se balade sur les colonnes
    for imageX in range(small_half_width, small_half_width + np.shape(image)[1]):
        # On se balade sur les lignes
        for imageY in range(small_half_width, small_half_width + np.shape(image)[0]):
            # On trouve la petite fenêtre de base
            patch_ref = padded_image[imageY - small_half_width : imageY + small_half_width + 1, imageX - small_half_width : imageX + small_half_width + 1]
            # On trouve les coordonnées inférieures de la première petite fenêtre comprise dans la grosse fenêtre
            bWinX, bWinY = big_patch(img = padded_image, coord_x = imageX , coord_y = imageY , size_w = big_window_size)
            denom = 0.0
            num = 0.0
            # On se balaye sur toutes les petites fenêtres possibles
            for sWinX in range(bWinX, bWinX + big_window_size - small_window_size + 1):
                for sWinY in range(bWinY, bWinY + big_window_size - small_window_size + 1):
                    moving_patch = padded_image[sWinY : sWinY + small_window_size ,sWinX : sWinX + small_window_size]
                    # On calcule le poids, qui résulte de la distance pondérée entre les deux patchs
                    weight = patch_distance_2d(p1 = patch_ref, p2 = moving_patch, s = small_window_size, w = w)
                    denom += weight
                    num += weight * padded_image[sWinY + small_half_width, sWinX + small_half_width]
                    # On affiche tous les 5 pourcents
                    iterator += 1
                    percent_complete = iterator*100/total_iterations
                    # On affiche tous les 5% d'itérations
                    if percent_complete % 5 == 0:
                        print('% COMPLETE = ', percent_complete,end='\r')

            # On assigne la valeur au pixel final
            output_image[imageY, imageX] = num / denom

    return output_image[small_half_width:small_half_width+np.shape(image)[0],small_half_width:small_half_width+np.shape(image)[1]]



La taille de la fenêtre influe sur le temps d'execution en $O(n^2)$. Pour accélerer le processus, on peut éventuellement choisir une taille de fenêtre relativement petite et appliquer plusieurs fois l'algorithme. 

In [ ]:
n_epochs=1
t=time.time()
im_denoised_array=im_noised_array.copy()
for _ in range(n_epochs):    
    im_denoised_array = non_local_means_denoising(im_denoised_array,
                        small_window_size = 7,
                        big_window_size = 20,
                        h = 8)
print('temps d\'exe',time.time()-t)
pn.Row(hv.Raster(im_denoised_array).opts(**options),hv.Raster(im_good_array).opts(**options),hv.Raster(im_noised_array).opts(**options))

In [ ]:
psnr=pd.DataFrame([PSNR(im_denoised_array, im_good_array)],columns=['PSNR image débruitée / image bruitée'])
display(psnr)

# Affichage des patchs voisins

Pour illustrer le fonctionnement de la méthode, on présente ci dessous un code qui affiche les patchs voisins du patch associé à un pixel donné.

In [ ]:
image=chargeData('carrelage')
im_good=Image.fromarray(image)
im_good = im_good.resize(new_size)
im_good_array = np.asarray(im_good)
im_noised_array=create_gaussian_noise(im_good_array,0,0.005)

In [ ]:
def fct_padded_image(image, small_window_size): # Retourne l'image avec du padding sur les cotés
    """
    image : image
    small_window_size : taille des patchs
    """
    # Les tailles des fenêtres doivent être impaires pour qu'elles soient centrées sur un pixel
    if small_window_size % 2 == 0:
        small_window_size += 1
    # On définit les moitiés de patchs
    small_half_width = small_window_size // 2
    # On crée une image avec du padding sur les cotés
    padded_image = np.ascontiguousarray(
        np.pad(image, ((small_half_width, small_half_width), (small_half_width, small_half_width)),
               mode='reflect').astype(np.float64))
    return padded_image

In [ ]:
def fct_coords_patch_ref(image, small_window_size , coord_pix): # Retourne les coordonnées du patch associé au pixel sélectionné
    """
    image : image
    small_window_size : taille des patchs
    coord_pix : coordonnées associées à l'image de base
    """
    # Les tailles des fenêtres doivent être impaires pour qu'elles soient centrées sur un pixel
    if small_window_size % 2 == 0:
        small_window_size += 1
    # On définit les moitiés de patchs
    small_half_width = small_window_size // 2
    # On trouve les coordonnées dans l'image avec du padding
    imageX = coord_pix[0] + small_window_size
    imageY = coord_pix[1] + small_window_size
    # On trouve les coordonnées du patch de référence
    coords = [[imageY - small_half_width , imageY + small_half_width + 1], [imageX - small_half_width , imageX + small_half_width + 1]]
    return coords

In [ ]:
def fct_coords_big_patch(image, coord_x, coord_y, size_w): # Retourne les coordonnées inférieures de la grosse fenêtre de recherche de patchs (retourne les coordonnées de l'angle en haut à gauche).
    """
    image : image
    coord_x : coordonnée x
    coord_y : coordonnée y
    size_w : taille de la grosse fenêtre
    """
    # On calcule les coordonnées de la grosse fenêtre de recherche
    b_inf_x = coord_x - (size_w//2)
    b_sup_x = coord_x + (size_w//2)
    b_inf_y = coord_y - (size_w//2)
    b_sup_y = coord_y + (size_w//2)
    # On gère les cas ou on déborde de l'image
    if b_inf_x < 0 :
        b_sup_x -= b_inf_x
        b_inf_x = 0
    elif b_sup_x >= np.shape(image)[1] :
        b_inf_x -= b_sup_x - (np.shape(image)[1] - 1)
        b_sup_x = np.shape(image)[1] - 1
    if b_inf_y < 0 :
        b_sup_y -= b_inf_y
        b_inf_y = 0
    elif b_sup_y >= np.shape(image)[0] :
        b_inf_y -= b_sup_y - (np.shape(image)[0] - 1)
        b_sup_y = np.shape(image)[0] - 1
    return [int(b_inf_y), int(b_sup_y+1)], [int(b_inf_x), int(b_sup_x+1)]

In [ ]:
def fct_coords_patch_top4(image, big_window_size, small_window_size , coord_pix, h = 40): # Retourne les 4 patchs les plus ressemblants au patch ref
    """
    image : image
    big_window_size : taille de la grosse fenêtre
    small_window_size : taille des patchs
    coord_pix : coordonnées associées à l'image de base
    h : parametre influençant le calcul des distances entre les patchs
    """
    # Les tailles des fenêtres doivent être impaires pour qu'elles soient centrées sur un pixel
    if small_window_size % 2 == 0:
        small_window_size += 1
    if big_window_size % 2 == 0:
        big_window_size += 1
    # On définit les moitiés de patchs
    small_half_width = small_window_size // 2
    big_half_width = big_window_size // 2
    # On crée une image avec du padding sur les cotés
    padded_image = np.ascontiguousarray(
        np.pad(image, ((small_half_width, small_half_width), (small_half_width, small_half_width)),
               mode='reflect').astype(np.float64))
    # On trouve les coordonnées dans l'image avec du padding
    imageX = coord_pix[0] + small_window_size
    imageY = coord_pix[1] + small_window_size
    t = small_window_size//2
    A = ((small_window_size - 1.) / 4.)
    range_vals = np.arange(-t, t + 1)
    xg_row, xg_col = np.meshgrid(range_vals, range_vals, indexing='ij')
    # On crée une matrice de poids w, qui pondère le calcul des distances entre patchs. Les pixels centraux des patchs sont ceux qui influencent le plus les distances entre patchs
    w = np.ascontiguousarray(np.exp(-(xg_row ** 2  + xg_col ** 2)/ (2 * A**2)))
    w *= 1. / (np.sum(w) * h**2)
    # On trouve les coordonnées inférieures de la première petite fenêtre comprise dans la grosse fenêtre
    bWinX, bWinY = big_patch(img = padded_image, coord_x = imageX , coord_y = imageY , size_w = big_window_size)
    # On trouve la petite fenêtre de base
    patch_ref = padded_image[imageY - small_half_width : imageY + small_half_width + 1, imageX - small_half_width : imageX + small_half_width + 1]
    coord_list = []
    weight_list = []
    # On balaye sur tous les patchs possibles
    for sWinX in range(bWinX, bWinX + big_window_size - small_window_size + 1):
        for sWinY in range(bWinY, bWinY + big_window_size - small_window_size + 1):
            # On vérifie que le patch ne soit pas le patch de base
            if not((sWinY+small_half_width == imageY) and (sWinX+small_half_width == imageX)) :
                # On trouve le patch
                moving_patch = padded_image[sWinY : sWinY + small_window_size ,sWinX : sWinX + small_window_size]
                # On calcule le poids, qui résulte de la distance pondérée entre les deux patchs
                weight = patch_distance_2d(p1 = patch_ref, p2 = moving_patch, s = small_window_size, w = w)
                # On ajoute le poids à la liste de poids
                weight_list.append(weight)
                # On trouve les coordonnées
                coord_list.append([[sWinY , sWinY + small_window_size ],[sWinX , sWinX + small_window_size]])
    # On trie les listes de manière décroissante selon les distances
    zipped_lists = list(zip(weight_list, coord_list))
    sorted_lists = sorted(zipped_lists, key=lambda x: x[0], reverse = True)
    sorted_weight_list, sorted_coord_list = zip(*sorted_lists)

    return sorted_coord_list[:4]

In [ ]:
# On choisit la taille de la grosse fenêtre de recherche, la taille des patchs ainsi que les coordonnées du pixel que l'on veut étudier
big_window_size = 101
small_window_size = 7
coord_pix = [35,27]

In [ ]:
padded_image = fct_padded_image(image = im_noised_array,
                                small_window_size = small_window_size)

coords_patch_ref = fct_coords_patch_ref(image = im_noised_array,
                                        small_window_size = small_window_size,
                                        coord_pix = coord_pix)

coords_big_patch = fct_coords_big_patch(image = padded_image,
                                        coord_x = coord_pix[0] + small_window_size ,
                                        coord_y = coord_pix[1] + small_window_size ,
                                        size_w = big_window_size)

coords_patch_top1, coords_patch_top2, coords_patch_top3, coords_patch_top4 = fct_coords_patch_top4(image = im_noised_array,
                                                                                           big_window_size = big_window_size,
                                                                                           small_window_size = small_window_size,
                                                                                           coord_pix = coord_pix)

In [ ]:
plt.imshow(padded_image,cmap='gray')

plt.plot([coords_big_patch[0][0],coords_big_patch[0][0]],[coords_big_patch[1][0],coords_big_patch[1][1]], color = 'g')
plt.plot([coords_big_patch[0][1],coords_big_patch[0][1]],[coords_big_patch[1][0],coords_big_patch[1][1]], color = 'g')
plt.plot([coords_big_patch[0][0],coords_big_patch[0][1]],[coords_big_patch[1][0],coords_big_patch[1][0]], color = 'g')
plt.plot([coords_big_patch[0][0],coords_big_patch[0][1]],[coords_big_patch[1][1],coords_big_patch[1][1]], color = 'g')

plt.plot([coords_patch_top1[0][0],coords_patch_top1[0][0]],[coords_patch_top1[1][0],coords_patch_top1[1][1]], color = 'r')
plt.plot([coords_patch_top1[0][1],coords_patch_top1[0][1]],[coords_patch_top1[1][0],coords_patch_top1[1][1]], color = 'r')
plt.plot([coords_patch_top1[0][0],coords_patch_top1[0][1]],[coords_patch_top1[1][0],coords_patch_top1[1][0]], color = 'r')
plt.plot([coords_patch_top1[0][0],coords_patch_top1[0][1]],[coords_patch_top1[1][1],coords_patch_top1[1][1]], color = 'r')

plt.plot([coords_patch_top2[0][0],coords_patch_top2[0][0]],[coords_patch_top2[1][0],coords_patch_top2[1][1]], color = 'r')
plt.plot([coords_patch_top2[0][1],coords_patch_top2[0][1]],[coords_patch_top2[1][0],coords_patch_top2[1][1]], color = 'r')
plt.plot([coords_patch_top2[0][0],coords_patch_top2[0][1]],[coords_patch_top2[1][0],coords_patch_top2[1][0]], color = 'r')
plt.plot([coords_patch_top2[0][0],coords_patch_top2[0][1]],[coords_patch_top2[1][1],coords_patch_top2[1][1]], color = 'r')

plt.plot([coords_patch_top3[0][0],coords_patch_top3[0][0]],[coords_patch_top3[1][0],coords_patch_top3[1][1]], color = 'r')
plt.plot([coords_patch_top3[0][1],coords_patch_top3[0][1]],[coords_patch_top3[1][0],coords_patch_top3[1][1]], color = 'r')
plt.plot([coords_patch_top3[0][0],coords_patch_top3[0][1]],[coords_patch_top3[1][0],coords_patch_top3[1][0]], color = 'r')
plt.plot([coords_patch_top3[0][0],coords_patch_top3[0][1]],[coords_patch_top3[1][1],coords_patch_top3[1][1]], color = 'r')

plt.plot([coords_patch_top4[0][0],coords_patch_top4[0][0]],[coords_patch_top4[1][0],coords_patch_top4[1][1]], color = 'r')
plt.plot([coords_patch_top4[0][1],coords_patch_top4[0][1]],[coords_patch_top4[1][0],coords_patch_top4[1][1]], color = 'r')
plt.plot([coords_patch_top4[0][0],coords_patch_top4[0][1]],[coords_patch_top4[1][0],coords_patch_top4[1][0]], color = 'r')
plt.plot([coords_patch_top4[0][0],coords_patch_top4[0][1]],[coords_patch_top4[1][1],coords_patch_top4[1][1]], color = 'r')

plt.plot(coord_pix[1]+ small_window_size,coord_pix[0]+ small_window_size,'*',color = 'b')
plt.plot([coords_patch_ref[0][0],coords_patch_ref[0][0]],[coords_patch_ref[1][0],coords_patch_ref[1][1]], color = 'b')
plt.plot([coords_patch_ref[0][1],coords_patch_ref[0][1]],[coords_patch_ref[1][0],coords_patch_ref[1][1]], color = 'b')
plt.plot([coords_patch_ref[0][0],coords_patch_ref[0][1]],[coords_patch_ref[1][0],coords_patch_ref[1][0]], color = 'b')
plt.plot([coords_patch_ref[0][0],coords_patch_ref[0][1]],[coords_patch_ref[1][1],coords_patch_ref[1][1]], color = 'b')

In [ ]:
patch_ref =  padded_image[coords_patch_ref[1][0]:coords_patch_ref[1][1],coords_patch_ref[0][0]:coords_patch_ref[0][1]]
patch_top1 = padded_image[coords_patch_top1[1][0]:coords_patch_top1[1][1],coords_patch_top1[0][0]:coords_patch_top1[0][1]]
patch_top2 = padded_image[coords_patch_top2[1][0]:coords_patch_top2[1][1],coords_patch_top2[0][0]:coords_patch_top2[0][1]]
patch_top3 = padded_image[coords_patch_top3[1][0]:coords_patch_top3[1][1],coords_patch_top3[0][0]:coords_patch_top3[0][1]]
patch_top4 = padded_image[coords_patch_top4[1][0]:coords_patch_top4[1][1],coords_patch_top4[0][0]:coords_patch_top4[0][1]]

pn.Row(hv.Raster(patch_ref).opts(**options),hv.Raster(patch_top1).opts(**options),hv.Raster(patch_top2).opts(**options),hv.Raster(patch_top3).opts(**options),hv.Raster(patch_top4).opts(**options))